In [48]:
# The magic commands below allow reflecting the changes in an imported module without restarting the kernel.
%load_ext autoreload
%autoreload 2

#!source /lus/theta-fs0/projects/connectomics_aesp/software/neuro_env_36/bin/activate-nomodules && LD_LIBRARY_PATH=/soft/interpreters/python/3.6/intel/2019.3.075/lib/:$LD_LIBRARY_PATH python -m ipykernel install --user --name jhub_neurenev36 && echo "success"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# We need to add balsam and the modules it depends on to the Python search paths. 
import sys
import os
from pprint import pprint
import matplotlib.pyplot
import balsam
from balsam.launcher.dag import submit

#Note to self..
#I don't like this..
sys.path.insert(0,'/lus/theta-fs0/projects/connectomics_aesp/software/HappyNeuron/')
##I really don't like this..
sys.path.insert(0,'/lus/theta-fs0/projects/connectomics_aesp/software/neuro_env_36/lib/python3.6/site-packages/')
##and this one..
os.environ['PATH'] +=':/soft/datascience/PostgreSQL/9.6.12/bin/'


import happyneuron as hpn
from  happyneuron.balsam.rvescovi_paths import *
import happyneuron.balsam
import happyneuron.balsam.hpn_balsam_apps as apps

In [58]:
# We also need to activate Balsam database by setting the BALSAM_DB_PATH environment variable. 
# This is equivalent to `source balsamactivate jupyter_test` 
database_path='/lus/theta-fs0/projects/connectomics_aesp/ravescovi/balsam_db/'
hpn.balsam.BalsamContext(database_path)

##there is some circularity here.. the upper cells fails until you run this one but this one fails if you don't run the 
##upper one..

AttributeError: module 'happyneuron' has no attribute 'balsam'

python /lus/theta-fs0/projects/connectomics_aesp/software/HappyNeuron/happyneuron/trakem2/mpi_montage.py
Created new app
python /lus/theta-fs0/projects/connectomics_aesp/software/HappyNeuron/happyneuron/trakem2/preprocess_stack.py
Updated existing app
python /lus/theta-fs0/projects/connectomics_aesp/software/HappyNeuron/happyneuron/trakem2/align.py
Updated existing app
python /lus/theta-fs0/projects/connectomics_aesp/software/HappyNeuron/happyneuron/trakem2/mpi_export.py
Updated existing app
python /lus/theta-fs0/projects/connectomics_aesp/software/HappyNeuron/happyneuron/aligntk/gen_mask.py
Updated existing app
python /lus/theta-fs0/projects/connectomics_aesp/software/HappyNeuron/happyneuron/aligntk/mpi_apply_map.py
Updated existing app
python /lus/theta-fs0/projects/connectomics_aesp/software/HappyNeuron/happyneuron/aligntk/find_rst.py
Updated existing app
python /lus/theta-fs0/projects/connectomics_aesp/software/HappyNeuron/happyneuron/aligntk/register.py
Updated existing app
python

# TRAKEM2 MONTAGE JOB

In [56]:
workflow = "trakem2_test_v4"
##
RAW_INPUT="/lus/theta-fs0/projects/connectomics_aesp/pipeline_data/trakem2_HL00732_run/trakem2_HL00732/"
PROCESS_FOLDER="/lus/theta-fs0/projects/connectomics_aesp/pipeline_data/trakem2_HL00732_run/trakem2_balsam6/"
TARGET_FOLDER =""
MIN=1000
MAX=1500
##
job = apps.trakem_montage_job(workflow, RAW_INPUT, PROCESS_FOLDER,target=TARGET_FOLDER, min=MIN, max=MAX, num_nodes=2)
print (job)

Input: /lus/theta-fs0/projects/connectomics_aesp/pipeline_data/trakem2_HL00732_run/trakem2_HL00732/
Output: /lus/theta-fs0/projects/connectomics_aesp/pipeline_data/trakem2_HL00732_run/trakem2_balsam6/align_raw.txt
(14000, 10833)
14000 10833 0 254 uint8
8bit
BalsamJob 7460ef01-4a18-44d0-befc-40a96020e05f
----------------------------------------------
workflow:                       trakem2_test_v4
name:                           montage
description:                    
lock:                           
parents:                        []
input_files:                    *
stage_in_url:                   
stage_out_files:                
stage_out_url:                  
wall_time_minutes:              1
num_nodes:                      2
coschedule_num_nodes:           0
ranks_per_node:                 4
cpu_affinity:                   depth
threads_per_rank:               1
threads_per_core:               1
node_packing_count:             1
environ_vars:                   OMP_NUM_THREADS=32

# TRAKEM2 ALIGN JOB

# TRAKEM2 SCRATCH SPACE

In [ ]:
# python $PRE_STACK $PROCESS_FOLDER/output $PROCESS_FOLDER/align_new.txt #input #output

#SERIAL SINGLE NODE OPERATION!!!
# $SUBMIT_THETA python $ALIGN $PROCESS_FOLDER/align_new.txt $PROCESS_FOLDER/align1 --fiji $FIJI

#$SUBMIT_THETA python $EXPORT $PROCESS_FOLDER/align_new.txt $PROCESS_FOLDER/align1 --range $RANGE --fiji $FIJI
#sleep 1



# FFN Compute Partitions

In [ ]:
workflow = 'ffn_comp_part_v1'
##
INPUT_VOLUME = '/lus/theta-fs0/projects/connectomics_aesp/pipeline_data/ffn_preprocess/data/sample_A_subset_ffn.h5'
INPUT_VOLUME_DSET = 'volumes/labels/neuron_ids_8'
OUTPUT_VOLUME = '/lus/theta-fs0/projects/connectomics_aesp/pipeline_data/ffn_preprocess/run1/sample_A_subset_parts.h5'
THRESHOLDS = '0.025,0.05,0.075,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9'
LOM_RADIUS = '5,5,5'
MIN_SIZE = '50'
##  
job = apps.ffn_compute_partitions_job(INPUT_VOLUME, INPUT_VOLUME_DSET, OUTPUT_VOLUME, THRESHOLDS, LOM_RADIUS, MIN_SIZE, workflow,num_nodes=4)
print (job)

# FFN Build Coordinates

In [ ]:
workflow = 'ffn_comp_part_v1'
##
TFRECORDFILE = '/lus/theta-fs0/projects/connectomics_aesp/pipeline_data/ffn_preprocess/run1/tf_record_file'
PARTITION_DATA = OUTPUT_VOLUME
SAMPLE = 'Cremi_A_subset'
MARGIN = '5,5,5'
##
job = apps.ffn_build_coordinates_job(SAMPLE, PARTITION_DATA, TFRECORDFILE, MARGIN, workflow, num_nodes=4)    
print(job)

# Flood Fill Network Training Job

In [ ]:
workflow = 'ffn_train'
##
TFRECORDFILE='/lus/theta-fs0/projects/datascience/keceli/run/f3n/training/tf_record_file'
GROUNDTRUTH='/lus/theta-fs0/projects/datascience/keceli/run/f3n/training/groundtruth.h5'
GRAYSCALE='/lus/theta-fs0/projects/datascience/keceli/run/f3n/training/grayscale_maps.h5'
BATCHSIZE=1
OPTIMIZER='adam'
TIMESTAMP=time.strftime("%y%m%d%H%M%S")
TRAINDIR=f'train_b{BATCHSIZE}_o{OPTIMIZER}_{TIMESTAMP}'
DEPTH = 12
FOV = "33, 33, 33" 
DELTA = "8, 8, 8"
##
job = apps.ffn_train_network_job(TFRECORDFILE, GROUNDTRUTH, GRAYSCALE, BATCHSIZE, OPTIMIZER, TIMESTAMP, TRAINDIR, DEPTH, FOV, DELTA, workflow)  
print(job)

# Flood Fill Network Inference Job

In [ ]:
workflow='ffn_parallel_inference'
##
INPUT_FILE = 'input_file_parallel.h5'
INPUT_DSET = 'image'
OUTPUT_FILE = 'output_file_parallel.h5'
OUTPUT_DSET = 'image'
OUTPUT_PATH = 'results_test_parallel6'
MEAN = 100
STD = 30
MODEL_PATH = 'model_vessel/model.ckpt-1680104'
DEPTH = 2
START = (0,0,0)
SIZE = (1000,1000,1000)
CHUNK_SIZE = (256,256,256)
OVERLAP = (16,16,16)
##
##cleanup all of the bellow to receive in a single function all of the above.
bbox = bounding_box.BoundingBox(start=START,size=SIZE)
pars =  (INPUT_FILE,INPUT_DSET, MEAN, STD, MODEL_PATH, OUTPUT_PATH)
##
test = create_inference_config(pars, 'test')
config_file = '/lus/theta-fs0/projects/connectomics_aesp/ravescovi/ffn_vessels_overlay_sean_anno/wholebrain.pbtxt'
##
boxes = divide_bounding_box(bbox,subvolume_size=CHUNK_SIZE,overlap=OVERLAP)
print(f'Number of subvolumes: {len(boxes)}')
##
jobs = apps.generate_balsam_inference_jobs(boxes, config_file, workflow_name='inference_8_8_v13')
print(jobs)

# BALSAM WORKFLOW SUBMISSION

In [35]:
# If you see 'Submit OK:', Job submission is succesful.

workflow = ''

submit(project='connectomics_aesp',
       queue='debug-flat-quad',
       nodes=1,
       job_mode='mpi',
       wall_minutes=40,
       wf_filter=workflow)


Submit OK: Qlaunch {   'command': '/lus/theta-fs0/projects/connectomics_aesp/ravescovi/balsam_db/qsubmit/qlaunch12.sh',
    'from_balsam': True,
    'id': 12,
    'job_mode': 'mpi',
    'nodes': 1,
    'prescheduled_only': False,
    'project': 'connectomics_aesp',
    'queue': 'debug-flat-quad',
    'scheduler_id': 386126,
    'state': 'submitted',
    'wall_minutes': 40,
    'wf_filter': ''}


# BALSAM WORKFLOW ANALISER

In [ ]:
# Balsam metadata
from balsam.core.models import utilization_report, throughput_report, process_job_times, BalsamJob
from matplotlib import pyplot as plt

workflow = "trakem2_test_v3"

##useful stuff
query = BalsamJob.objects.filter(workflow=workflow)
time_dat = process_job_times(query) #filters into a single workflow
[j.runtime_seconds for j in query] # full time per balsam Job

times_created, num_created = sorted(time_dat['CREATED']), range(1, len(time_dat["CREATED"])+1)

t0 = min(times_created)

def mins(t):
    return (t-t0).total_seconds() / 60

plt.step([mins(t) for t in times_created] ,num_created, 'o', where='post',label='creation')
times, num_thru = throughput_report(time_dat)
plt.step([mins(t) for t in times], num_thru,  'o', where='post', label='done')

times_u, num_util = utilization_report(time_dat)
plt.step([mins(t) for t in times_u], num_util, 'o', where='post', label='utilization')
plt.legend()

# Flood Fill Network MultScale Training Job

In [ ]:
#multiscale training job
TFRECORDFILE='/lus/theta-fs0/projects/datascience/keceli/run/f3n/training/tf_record_file'
GROUNDTRUTH='/lus/theta-fs0/projects/datascience/keceli/run/f3n/training/groundtruth.h5'
GRAYSCALE='/lus/theta-fs0/projects/datascience/keceli/run/f3n/training/grayscale_maps.h5'
BATCHSIZE=1
OPTIMIZER='adam'

MAGS = [1,2,4,8,16]

##DOWNSAMPLE INPUT DATA AND ORGANIZE DIRS
#for mag in MAGS:
    
    
for mag in MAGS:    
    TRAINDIR=f'train_b{BATCHSIZE}_o{OPTIMIZER}_m{mag}_{TIMESTAMP}'
    myargs = ''
    myargs += f' --train_coords {TFRECORDFILE} '
    myargs += f' --data_volumes valdation1:{GRAYSCALE}:raw '
    myargs += f' --label_volumes valdation1:{GROUNDTRUTH}:stack '
    myargs += f' --model_name convstack_3d.ConvStack3DFFNModel '
    myargs += ''' --model_args "{\\"depth\\": 12, \\"fov_size\\": [33, 33, 33], \\"deltas\\": [8, 8, 8]}"'''
    myargs += ' --image_mean 128 --image_stddev 33 '
    myargs += ' --max_steps 40000000 --summary_rate_secs 360 ' 
    myargs += f' --batch_size {BATCHSIZE} '
    myargs += f' --optimizer {OPTIMIZER} '
    myargs += ' --num_intra_threads 64 --num_inter_threads 1 '
    myargs += f' --train_dir {TRAINDIR} '

    add_job(name=f'train_mag{mag}',
            workflow='ffn_training',
            application='trainer',
            args=myargs,
            ranks_per_node=rpn,
            num_nodes=nnode,
            environ_vars={'OMP_NUM_THREADS=64'})

# Scratch Space Bellow

In [ ]:
# Adding a training job to the database
import time
TFRECORDFILE='/lus/theta-fs0/projects/datascience/keceli/run/f3n/training/tf_record_file'
GROUNDTRUTH='/lus/theta-fs0/projects/datascience/keceli/run/f3n/training/groundtruth.h5'
GRAYSCALE='/lus/theta-fs0/projects/datascience/keceli/run/f3n/training/grayscale_maps.h5'
BATCHSIZE=1
OPTIMIZER='adam'
TIMESTAMP=time.strftime("%y%m%d%H%M%S")
for rpn in [1,4,16]:
    for nnode in [1,4,16,64]:
        TRAINDIR=f'train_b{BATCHSIZE}_o{OPTIMIZER}_n{nnode}_r{rpn}_{TIMESTAMP}'
        myargs = ''
        myargs += f' --train_coords {TFRECORDFILE} '
        myargs += f' --data_volumes valdation1:{GRAYSCALE}:raw '
        myargs += f' --label_volumes valdation1:{GROUNDTRUTH}:stack '
        myargs += f' --model_name convstack_3d.ConvStack3DFFNModel '
        myargs += ''' --model_args "{\\"depth\\": 12, \\"fov_size\\": [33, 33, 33], \\"deltas\\": [8, 8, 8]}"'''
        myargs += ' --image_mean 128 --image_stddev 33 '
        myargs += ' --max_steps 40000000 --summary_rate_secs 360 ' 
        myargs += f' --batch_size {BATCHSIZE} '
        myargs += f' --optimizer {OPTIMIZER} '
        myargs += ' --num_intra_threads 64 --num_inter_threads 1 '
        myargs += f' --train_dir {TRAINDIR} '

        add_job(name=f'train_n{nnode}_r{rpn}',
                workflow='ffn_training',
                application='trainer',
                args=myargs,
                ranks_per_node=rpn,
                num_nodes=nnode,
                environ_vars={'OMP_NUM_THREADS=64'})

In [6]:
##ALIGNTK APPS 

from glob import glob

##TODO jobalize the layers bellow

#!/bin/bash
#COBALT -t 180
#COBALT -n 128
#COBALT -q default
#COBALT -A connectomics_aesp

# NODES=$COBALT_JOBSIZE
# PROC_PER_NODE=64
# PROC=$((NODES * PROC_PER_NODE))
PROC=777

ALIGNTK_DIR="/lus/theta-fs0/projects/connectomics_aesp/software/aligntk-1.0.2/install/bin"
IMAGE_DIR="/lus/theta-fs0/projects/connectomics_aesp/pipeline_data/trakem2_HL00732_run/trakem2_HL00732_process_1/output/"
MASK_DIR="/lus/theta-fs0/projects/connectomics_aesp/pipeline_data/aligntk_HL00732/out1_mask"
OUTPUT_DIR="/lus/theta-fs0/projects/connectomics_aesp/pipeline_data/aligntk_HL00732/out1"
GROUP_SIZE=PROC-1
N_IMAGES= len(glob(IMAGE_DIR+'*'))
N_GROUPS=((N_IMAGES / GROUP_SIZE + 1))


ModuleNotFoundError: No module named 'happyneuron'

In [ ]:
print("Total image count: "+str(N_IMAGES))
print("Number of groups: "+str(N_GROUPS))

#aligntk_create_dirs(OUTPUT_DIR, MASK_DIR)
#make_schedule(OUTPUT_DIR)
#preprocess_main(IMAGE_DIR, OUTPUT_DIR, 12)

from happyneuron.aligntk.preprocess import *

def aligntk_preprocess_job(image_dir, mask_dir, low, high, kernel, workflow)


  #aprun -n 777 -N $PROC_PER_NODE python -m klab_utils.aligntk_gen_mask --image_dir $IMAGE_DIR --mask_dir $MASK_DIR --low 10 --high 240 --kernel 10
      pass

In [ ]:

add_app(name='aligntk_findrst',
        executable='python /lus/theta-fs0/projects/connectomics_aesp/software/HappyNeuron/happyneuron/aligntk/find_rst.py',
        description='AlignTK Apply Map',
        envscript=env_preamble)

def aligntk_rst_job( aligntk=''):

    
    for i in np.arange(N_GROUPS):
#  aprun -n $PROC -N $PROC_PER_NODE $ALIGNTK_DIR/find_rst -pairs pairs$i.lst -tif -images $IMAGE_DIR -mask $MASK_DIR -output $OUTPUT_DIR/cmaps/ -rotation -15-15 -max_res 8192 -scale 0.8-1.2 -tx -30-30 -ty -30-30 -summary $OUTPUT_DIR/cmaps/summary$i.out

    pass

In [ ]:
def aligntk_register_job():
#for (( i=0; i<$N_GROUPS; i++ ))

#  aprun -n $PROC -N $PROC_PER_NODE $ALIGNTK_DIR/register -pairs pairs$i.lst -images $IMAGE_DIR -mask $MASK_DIR -tif -output $OUTPUT_DIR/maps/ -distortion 6.0 -output_level 6 -depth 6 -quality 0.5 -summary $OUTPUT_DIR/maps/summary$i.out -initial_map $OUTPUT_DIR/cmaps/
##
#aprun -n $PROC -N $PROC_PER_NODE $ALIGNTK_DIR/register -pairs pairs.lst -images $IMAGE_DIR -mask $MASK_DIR -tif -output $OUTPUT_DIR/maps/ -distortion 4.0 -output_level 6 -depth 6 -quality 0.3 -summary $OUTPUT_DIR/maps/summary.out -initial_map $OUTPUT_DIR/cmaps/
    pass

In [ ]:
def aligntk_align_job():
#FIXED=`head images.lst -n 1`
# aprun -n 8192 -N 64 $ALIGNTK_DIR/align -images $IMAGE_DIR -image_list images.lst -map_list pairs.lst -maps $OUTPUT_DIR/maps/ -masks $MASK_DIR -output $OUTPUT_DIR/amaps/ -schedule schedule.lst -incremental -output_grid $OUTPUT_DIR/grids/ -grid_size 8192x8192 -fold_recovery 60
#aprun -n $PROC -N $PROC_PER_NODE $ALIGNTK_DIR/align -images $IMAGE_DIR -image_list images.lst -map_list pairs.lst -masks $MASK_DIR -maps $OUTPUT_DIR/maps/ -output $OUTPUT_DIR/amaps/ -schedule schedule.lst -incremental -output_grid $OUTPUT_DIR/grids/ -grid_size 8192x8192 -fold_recovery 60
    pass

In [ ]:
def aligntk_applymap_job():
##$ALIGNTK_DIR/apply_map -image_list images.lst -images $IMAGE_DIR -maps $OUTPUT_DIR/amaps/ -output $OUTPUT_DIR/aligned/ -memory 150000
#aprun -n 777 -N 64 python -m klab_utils.aligntk_mpi_apply_map ./outputs_v1/ --image_dir ../data/images_corr_v2/ --image_lst ./images.lst
    pass